In [4]:
import os
import requests
from src.dataset import load_shapes_and_records, max_inner_rect, get_soil_data
from src.util import Const
from tqdm.notebook import tnrange

In [5]:
shapes, records = load_shapes_and_records()
soil = get_soil_data()

In [6]:
labels = []
for i in tnrange(len(records)):
    if int(records[i].MUKEY) not in soil[Const.DB_MU_ID].values:
        continue

    humus_concentration = soil[soil[Const.DB_MU_ID] == int(records[i].MUKEY)][Const.DB_OM].values[0]

    bbox = max_inner_rect(shapes[i]).exterior.coords
    llx = bbox[0][0]
    lly = bbox[0][1]
    urx = bbox[2][0]
    ury = bbox[2][1]

    image_url = f'https://static-maps.yandex.ru/1.x/?l=sat&bbox={llx}%2C{lly}~{urx}%2C{ury}'
    response = requests.get(image_url)
    if response.status_code == requests.codes.OK:
        with open(os.path.join(Const.IMAGES_PATH, f'{i}.jpg'), 'wb') as image_file:
            image_file.write(response.content)

        labels.append(f'{i}.jpg:{humus_concentration}')

  0%|          | 0/1482 [00:00<?, ?it/s]

In [8]:
if len(labels) > 0:
    with open(Const.LABELS_FILE_PATH, 'w') as labels_file:
        for label in labels:
            labels_file.write(f'{label}\n')